In [1]:
#!pip install keras-tuner --upgrade
#!pip install tensorflow

In [3]:
from tensorflow import keras
from keras import backend as K
import keras_tuner as kt
from keras.layers import Layer
from keras.utils.vis_utils import plot_model
from tensorflow.keras import layers
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import SGD, Adam
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger

import os
import pandas as pd
import pickle
import datetime
import numpy as np

from matplotlib import pyplot as plt

## Prepare the data

In [4]:
# Model / data parameters
dataset = "MNIST"
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

if dataset == "CIFAR10":
    input_shape = (32, 32, 3)
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=0.20, shuffle=True
)

print(dataset)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_valid.shape[0], "validation samples")
print(x_test.shape[0], "test samples")

MNIST
x_train shape: (48000, 28, 28, 1)
48000 train samples
12000 validation samples
10000 test samples


## Building the MLP model

##### Building a custom layer for our random parameters

In [5]:
import random
import logging

# Constraint to fix/freeze all layer weights but some (two params per layer unit)
class FixWeights(tf.keras.constraints.Constraint):
    def __init__(self, shape):
        self.shape = shape
        self.init_weights = tf.keras.initializers.HeNormal(seed=None)(
            shape=self.shape
        ).numpy()
        self.trainable_weights_0 = []
        self.trainable_weights_1 = []
        rand_ints = random.sample(
            range(0, self.shape[0] * self.shape[1]), 2 * self.shape[1]
        )
        # Map random ints to coordinates
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                if i * self.shape[1] + j in rand_ints:
                    self.trainable_weights_0.append(i)
                    self.trainable_weights_1.append(j)

    def __call__(self, w):
        new_w = np.copy(self.init_weights)
        for i in range(len(self.trainable_weights_0)):
            new_w[self.trainable_weights_0[i]][self.trainable_weights_1[i]] = w[
                self.trainable_weights_0[i]
            ][self.trainable_weights_1[i]].numpy()

        return tf.convert_to_tensor(new_w)

In [6]:
class CustomRandomLayer(Layer):
    def __init__(self, trainable=True):
        self.trainable = trainable
        super(CustomRandomLayer, self).__init__()

    def build(self, input_shape):
        # Adding two weights with shape as number of channels to make two params per channel
        self.w1 = self.add_weight(
            name="w1",
            shape=(input_shape[-1],),
            initializer="he_normal",
            trainable=self.trainable,
        )
        self.w2 = self.add_weight(
            name="w1",
            shape=(input_shape[-1],),
            initializer="he_normal",
            trainable=self.trainable,
        )

    def call(self, x):
        return (x * self.w1) * self.w2

In [7]:
def fc_model_builder(
    units_dense_layer=10,
    dense_layers=1,
    dense=True,
    bn=False,
    random=False,
    dense_out=False,
):
    """
    Wrapper for constructing models with different trainable layers and units in dense layers.
    """
    model = keras.Sequential()
    model.add(keras.Input(shape=input_shape))
    model.add(layers.Flatten())

    # Adding variable amount of layers
    for i in range(0, dense_layers):
        model.add(
            layers.Dense(
                units_dense_layer,
                activation=None,
                kernel_initializer="he_normal",
                use_bias=random,
                trainable=dense,
                kernel_constraint=FixWeights(
                    shape=(model.layers[-1].output_shape[1], units_dense_layer)
                )
                if random
                else None,
            )
        )
        if bn:
            model.add(
                layers.BatchNormalization(
                    beta_initializer="zeros",
                    gamma_initializer=RandomNormal(mean=0.0, stddev=1.0),
                    trainable=True,
                )
            )
        if random:
            # model.add(CustomRandomLayer(trainable=True))

            model.add(
                layers.BatchNormalization(
                    beta_initializer="zeros",
                    gamma_initializer="ones",
                    trainable=False,
                )
            )

        model.add(layers.Activation("relu"))

    # Output layer
    model.add(
        layers.Dense(
            num_classes,
            activation="softmax",
            kernel_initializer="he_normal",
            trainable=dense_out,
        )
    )

    return model

In [ ]:
def fc_model_builder_dim_wrapper(
    layer_width=784, min_hdl=2, max_hdl=4, num_hdl_interval=1
):
    """
    Helper function for creating the models of interest with widht and depth as variables
    """
    fc_models = {}

    # Build nested dict with variable number of layered models
    for i in range(min_hdl, max_hdl + 1, num_hdl_interval):
        fc_models[i] = {}

        model_fc_all_layers = fc_model_builder(
            units_dense_layer=layer_width,
            dense_layers=i,
            dense=True,
            bn=True,
            random=False,
            dense_out=True,
        )
        vanilla_fc_model = fc_model_builder(
            units_dense_layer=layer_width,
            dense_layers=i,
            dense=True,
            bn=False,
            random=False,
            dense_out=True,
        )
        model_fc_bn = fc_model_builder(
            units_dense_layer=layer_width,
            dense_layers=i,
            dense=False,
            bn=True,
            random=False,
            dense_out=False,
        )
        model_fc_bn_out = fc_model_builder(
            units_dense_layer=layer_width,
            dense_layers=i,
            dense=False,
            bn=True,
            random=False,
            dense_out=True,
        )
        model_fc_random = fc_model_builder(
            units_dense_layer=layer_width,
            dense_layers=i,
            dense=True,
            bn=False,
            random=True,
            dense_out=False,
        )
        model_fc_out = fc_model_builder(
            units_dense_layer=layer_width,
            dense_layers=i,
            dense=False,
            bn=False,
            random=False,
            dense_out=True,
        )
        model_fc_none = fc_model_builder(
            units_dense_layer=layer_width,
            dense_layers=i,
            dense=False,
            bn=False,
            random=False,
            dense_out=False,
        )

        # Save models in dict
        fc_models[i] = {
            "model_fc_all_layers": model_fc_all_layers,
            #"vanilla_fc_model": vanilla_fc_model,
            "model_fc_bn": model_fc_bn,
            #"model_fc_bn_out": model_fc_bn_out,
            "model_fc_random": model_fc_random,
            #"model_fc_out": model_fc_out,
            #"model_fc_none": model_fc_none,
        }

    return fc_models

In [ ]:
# Creating a custom callback to change the learning rate at 70 to 100 epochs.
def scheduler(epoch, lr):
    if epoch in [70, 100]:
        return lr * 0.1
    return lr

"""
step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [28125, 42185], [1e-0, 1e-1, 1e-2]
)

wd = lambda: 1e-4 * schedule(step)
"""

In [36]:
# Creating callbacks
log_dir = os.path.join("logs/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        write_graph=True,
        write_images=True,
    )
]
callbacks.append(CSVLogger(log_dir + "/" + "latest.csv"))
callbacks.append(
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=3, restore_best_weights=True
    )
)
callbacks.append(keras.callbacks.LearningRateScheduler(scheduler, verbose=1))

## Train models

In [37]:
# Building models of different depth and trainable layers
fc_models = fc_model_builder_dim_wrapper(
    layer_width=10, min_hdl=2, max_hdl=14, num_hdl_interval=2
)

In [38]:
# Init training params
epochs = 80
batch_size = 128
model_histories = {}

# Create new time-based log dir
log_dir = os.path.join(
    "logs/dense",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
)

# Train all models in dict
for depth, nested in fc_models.items():
    model_histories[depth] = {}
    log_dir_depth = log_dir + "/" + str(depth)

    for key, model in nested.items():
        # Reinitialize callbacks with new directory
        log_dir_depth_key = log_dir_depth + "/" + key

        callbacks[0] = keras.callbacks.TensorBoard(
            log_dir=log_dir_depth_key,
            histogram_freq=1,
            write_graph=True,
            write_images=True,
        )
        callbacks[1] = CSVLogger(log_dir + "fit_csv_logger.csv")

        # Compile and fit models
        model.compile(
            loss=keras.losses.categorical_crossentropy,
            optimizer=SGD(learning_rate=0.01, momentum=0.9),
            metrics=["accuracy"],
            run_eagerly=True,
        )
        model_histories[depth][key] = model.fit(
            x_train,
            y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_valid, y_valid),
            callbacks=callbacks,
        )


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/80
375/375 [==============================] - 7s 19ms/step - loss: 0.8874 - accuracy: 0.7380 - val_loss: 0.4343 - val_accuracy: 0.8742 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/80
375/375 [==============================] - 9s 24ms/step - loss: 0.3734 - accuracy: 0.8956 - val_loss: 0.3256 - val_accuracy: 0.9056 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 3/80
375/375 [==============================] - 8s 21ms/step - loss: 0.3031 - accuracy: 0.9135 - val_loss: 0.2876 - val_accuracy: 0.9176 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 4/80
375/375 [==============================] - 6s 17ms/step - loss: 0.2738 - accuracy: 0.9215 - val_loss: 0.2669 - val_accuracy: 0.9228 - lr: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.0

375/375 [==============================] - 4s 10ms/step - loss: 0.2331 - accuracy: 0.9322 - val_loss: 0.2449 - val_accuracy: 0.9281 - lr: 0.0100

Epoch 9: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 9/80
375/375 [==============================] - 4s 11ms/step - loss: 0.2277 - accuracy: 0.9335 - val_loss: 0.2345 - val_accuracy: 0.9301 - lr: 0.0100

Epoch 10: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 10/80
375/375 [==============================] - 4s 11ms/step - loss: 0.2204 - accuracy: 0.9354 - val_loss: 0.2409 - val_accuracy: 0.9305 - lr: 0.0100

Epoch 11: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 11/80
375/375 [==============================] - 4s 11ms/step - loss: 0.2167 - accuracy: 0.9373 - val_loss: 0.2283 - val_accuracy: 0.9309 - lr: 0.0100

Epoch 12: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 12/80
375/375 [==============================] - 4s 11ms/ste

Epoch 6/80
375/375 [==============================] - 7s 18ms/step - loss: 1.6080 - accuracy: 0.4480 - val_loss: 1.5852 - val_accuracy: 0.4577 - lr: 0.0100

Epoch 7: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 7/80
375/375 [==============================] - 7s 18ms/step - loss: 1.5888 - accuracy: 0.4542 - val_loss: 1.5694 - val_accuracy: 0.4629 - lr: 0.0100

Epoch 8: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 8/80
375/375 [==============================] - 7s 18ms/step - loss: 1.5778 - accuracy: 0.4559 - val_loss: 1.5604 - val_accuracy: 0.4675 - lr: 0.0100

Epoch 9: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 9/80
375/375 [==============================] - 7s 18ms/step - loss: 1.5671 - accuracy: 0.4627 - val_loss: 1.5502 - val_accuracy: 0.4691 - lr: 0.0100

Epoch 10: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 10/80
375/375 [==============================] - 7s 1


Epoch 18: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 18/80
375/375 [==============================] - 10s 26ms/step - loss: 2.1122 - accuracy: 0.2659 - val_loss: 2.1174 - val_accuracy: 0.2583 - lr: 0.0100

Epoch 19: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 19/80
375/375 [==============================] - 10s 26ms/step - loss: 2.1118 - accuracy: 0.2659 - val_loss: 2.1170 - val_accuracy: 0.2594 - lr: 0.0100

Epoch 20: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 20/80
375/375 [==============================] - 10s 26ms/step - loss: 2.1112 - accuracy: 0.2648 - val_loss: 2.1167 - val_accuracy: 0.2583 - lr: 0.0100

Epoch 21: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 21/80
375/375 [==============================] - 9s 25ms/step - loss: 2.1109 - accuracy: 0.2671 - val_loss: 2.1161 - val_accuracy: 0.2603 - lr: 0.0100

Epoch 22: LearningRateScheduler setting learning

375/375 [==============================] - 4s 10ms/step - loss: 1.8942 - accuracy: 0.3304 - val_loss: 1.8848 - val_accuracy: 0.3377 - lr: 0.0100

Epoch 14: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 14/80
375/375 [==============================] - 4s 10ms/step - loss: 1.8906 - accuracy: 0.3313 - val_loss: 1.8814 - val_accuracy: 0.3396 - lr: 0.0100

Epoch 15: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 15/80
375/375 [==============================] - 4s 10ms/step - loss: 1.8874 - accuracy: 0.3321 - val_loss: 1.8781 - val_accuracy: 0.3392 - lr: 0.0100

Epoch 16: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 16/80
375/375 [==============================] - 4s 10ms/step - loss: 1.8843 - accuracy: 0.3321 - val_loss: 1.8754 - val_accuracy: 0.3408 - lr: 0.0100

Epoch 17: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 17/80
375/375 [==============================] - 4s 11ms/s

...

375/375 [==============================] - 6s 16ms/step - loss: 2.0118 - accuracy: 0.2828 - val_loss: 2.0087 - val_accuracy: 0.2863 - lr: 0.0100

Epoch 29: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 29/80
375/375 [==============================] - 5s 15ms/step - loss: 2.0103 - accuracy: 0.2852 - val_loss: 2.0064 - val_accuracy: 0.2852 - lr: 0.0100

Epoch 30: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 30/80
375/375 [==============================] - 6s 15ms/step - loss: 2.0087 - accuracy: 0.2849 - val_loss: 2.0060 - val_accuracy: 0.2799 - lr: 0.0100

Epoch 31: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 31/80
375/375 [==============================] - 5s 14ms/step - loss: 2.0078 - accuracy: 0.2851 - val_loss: 2.0061 - val_accuracy: 0.2910 - lr: 0.0100

Epoch 32: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 32/80
375/375 [==============================] - 5s 14ms/s


Epoch 4: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 4/80
375/375 [==============================] - 5s 14ms/step - loss: 2.5352 - accuracy: 0.0993 - val_loss: 2.5455 - val_accuracy: 0.0957 - lr: 0.0100

Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/80
375/375 [==============================] - 21s 57ms/step - loss: 1.6922 - accuracy: 0.3911 - val_loss: 1.2086 - val_accuracy: 0.5780 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/80
375/375 [==============================] - 20s 54ms/step - loss: 0.9910 - accuracy: 0.6534 - val_loss: 1.0371 - val_accuracy: 0.5996 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 3/80
375/375 [==============================] - 26s 70ms/step - loss: 0.7350 - accuracy: 0.7639 - val_loss: 0.6107 - val_accuracy: 0.8100 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 

375/375 [==============================] - 7s 18ms/step - loss: 0.4752 - accuracy: 0.8701 - val_loss: 0.4686 - val_accuracy: 0.8745 - lr: 0.0100

Epoch 6: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 6/80
375/375 [==============================] - 7s 19ms/step - loss: 0.4354 - accuracy: 0.8824 - val_loss: 0.4479 - val_accuracy: 0.8797 - lr: 0.0100

Epoch 7: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 7/80
375/375 [==============================] - 7s 18ms/step - loss: 0.4051 - accuracy: 0.8922 - val_loss: 0.3903 - val_accuracy: 0.8977 - lr: 0.0100

Epoch 8: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 8/80
375/375 [==============================] - 7s 19ms/step - loss: 0.3821 - accuracy: 0.8968 - val_loss: 0.4076 - val_accuracy: 0.8892 - lr: 0.0100

Epoch 9: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 9/80
375/375 [==============================] - 7s 20ms/step - lo

Epoch 6/80
375/375 [==============================] - 18s 49ms/step - loss: 1.9119 - accuracy: 0.2993 - val_loss: 1.9038 - val_accuracy: 0.2988 - lr: 0.0100

Epoch 7: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 7/80
375/375 [==============================] - 19s 52ms/step - loss: 1.8876 - accuracy: 0.3063 - val_loss: 1.8577 - val_accuracy: 0.3157 - lr: 0.0100

Epoch 8: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 8/80
375/375 [==============================] - 22s 58ms/step - loss: 1.8694 - accuracy: 0.3139 - val_loss: 1.8449 - val_accuracy: 0.3164 - lr: 0.0100

Epoch 9: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 9/80
375/375 [==============================] - 24s 64ms/step - loss: 1.8593 - accuracy: 0.3202 - val_loss: 1.8517 - val_accuracy: 0.3192 - lr: 0.0100

Epoch 10: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 10/80
375/375 [==============================] - 

KeyboardInterrupt: 

In [25]:
%reload_ext tensorboard

%tensorboard --logdir './logs/dense'

## Evaluate model

In [ ]:
# Evaluate on test dataset
loss_test_dict = {}
acc_test_dict = {}

for depth, nested in fc_models.items():
    acc_test_dict[depth] = {}
    loss_test_dict[depth] = {}
    for key, model in nested.items():
        score = model.evaluate(x_test, y_test, verbose=0)
        loss_test_dict[depth][key] = score[0]
        acc_test_dict[depth][key] = score[1]

In [ ]:
# Turn dict with acc metrics into dataframe and export
acc_test_frame = pd.DataFrame.from_dict(acc_test_dict, orient="index")
loss_test_frame = pd.DataFrame.from_dict(loss_test_dict, orient="index")

In [ ]:
acc_test_frame

In [ ]:
# Save statistics
acc_test_frame.to_csv(
    "logs/dense_test_stats_10_14_100_4"
    + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    + ".csv",
    index=True,
    index_label="layers",
)

## Visualize metrics

In [19]:
%reload_ext tensorboard

%tensorboard --logdir ./logs/

In [ ]:
for history in fc_model_histories:
    plt.plot(history.history["accuracy"])
    plt.plot(history.history["val_accuracy"])

plt.title("Model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="upper left")
plt.savefig("fc_model_accuracy.png")
plt.show()

## Save models

In [ ]:
# Save all models
for depth, nested in fc_models.items():
    for key, model in nested.items():
        model.save("models/saved764/" + str(depth) + key)

In [ ]:
# model = keras.models.load_model('path/to/location')

## Tune model

In [ ]:
# model = keras.models.load_model('path/to/location')

In [ ]:
for depth, nested in fc_models.items():
    for key, model in nested.items():
        # model_histories[key]
        hist_df = pd.DataFrame(model_histories[key].history)
        # or save to csv:
        hist_csv_file = "history.csv"
        with open(hist_csv_file, mode="w") as f:
            hist_df.to_csv(f)

        break

In [ ]:
class MyFcHyperModel(kt.HyperModel):
    def build(self, hp):
        model = fc_model_builder(
            dense=hp.Boolean("dense"),
            bn=hp.Boolean("bn"),
            random=not hp.Boolean("bn"),
            dense_out=hp.Boolean("output"),
        )
        model.compile(
            loss=keras.losses.categorical_crossentropy,
            optimizer=SGD(learning_rate=0.1, momentum=0.9),
            metrics=["accuracy"],
        )

        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=128,  # hp.Choice("batch_size", [16, 32, 64, 128])
            **kwargs,
        )

In [ ]:
fc_tuner = kt.RandomSearch(
    MyFcHyperModel(),
    objective="val_accuracy",
    max_trials=20,
    overwrite=True,
    # Set a directory to store the intermediate results.
    directory="/tmp/tb",
    project_name="fc_batchnorm",
)

In [ ]:
# fc_tuner.search(x_train, y_train, epochs=2, validation_data=(x_test, y_test), callbacks=[callbacks],)

In [ ]:
%reload_ext tensorboard

%tensorboard --logdir ./logs/fc